In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing.data import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing


columns = ['dataset','building','has_act','start','end'] + range(122)
data = pd.read_csv('windat_fridge.csv', names=columns, index_col=None, 
                   header=None, parse_dates=['start','end'], date_parser=pd.to_datetime)
data.head()

,dataset,building,has_act,start,end,0,1,2,3,4,...,112,113,114,115,116,117,118,119,120,121
0,redd,1,0,2011-04-18 16:02:48,2011-04-18 16:17:48,197.800003,194.679993,195.399994,195.089996,194.750000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,redd,1,0,2011-04-18 16:04:05,2011-04-18 16:19:05,195.089996,194.750000,194.679993,195.850006,195.559998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,redd,1,0,2011-04-18 16:04:10,2011-04-18 16:19:10,195.089996,194.750000,194.679993,195.850006,195.559998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,redd,1,0,2011-04-18 16:04:25,2011-04-18 16:19:25,195.089996,194.750000,194.679993,195.850006,195.559998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,redd,1,0,2011-04-18 16:05:04,2011-04-18 16:20:04,194.679993,195.850006,195.559998,195.880005,194.970001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# separar por dataset
data_redd = data[data['dataset'] == 'redd']
data_ukdale = data[data['dataset'] == 'ukdale']

# separar por si tienen o no activaciones
data_redd_act = data_redd[data_redd['has_act'] == 1]
data_redd_noact = data_redd[data_redd['has_act'] == 0]
data_ukdale_act = data_ukdale[data_ukdale['has_act'] == 1]
data_ukdale_noact = data_ukdale[data_ukdale['has_act'] == 0]

print "Casas de REDD con activaciones: " + str(data_redd_act['building'].unique())
print "Casas de REDD sin activaciones: " + str(data_redd_noact['building'].unique())
print "Casas de UK-DALE con activaciones: "+ str(data_ukdale_act['building'].unique())
print "Casas de UK-DALE sin activaciones: " + str(data_ukdale_noact['building'].unique())

Casas de REDD con activaciones: []
Casas de REDD sin activaciones: [1 2 3 5 6]
Casas de UK-DALE con activaciones: [1]
Casas de UK-DALE sin activaciones: [1 2 4 5]


Elijo la casas de REDD 5-6 y UKDALE-5 para validar como casas no vistas.

In [17]:
data_train = data_redd_act[data_redd_act['building'].isin([1,2,3])]
data_train = data_train.append(data_ukdale_act[data_ukdale_act['building'].isin([1,2,4])], ignore_index=True)
data_train = data_train.append(data_ukdale_noact, ignore_index=True)
data_valid = data_redd_act[data_redd_act['building'].isin([5,6])]
data_valid = data_valid.append(data_ukdale_noact[data_ukdale_noact['building'].isin([5])], ignore_index=True)

In [18]:
data_train = data_train[range(122)]
data_train = data_train.fillna(0)
data_valid = data_valid[range(122)]
data_valid = data_valid.fillna(0)

d = np.shape(data_train)[1]/2
X_train = preprocessing.scale(np.array(data_train.iloc[:,:d]), with_mean=False, with_std=True)
y_train = preprocessing.scale(np.array(data_train.iloc[:,d:]), with_mean=False, with_std=True)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

X_valid = preprocessing.scale(np.array(data_valid.iloc[:,:d]), with_mean=False, with_std=True)
y_valid = preprocessing.scale(np.array(data_valid.iloc[:,d:]), with_mean=False, with_std=True)

In [19]:
regressor = MLPRegressor(solver='adam',
                        alpha=1e-5,
                        activation = 'tanh',
                        verbose = False,
                        hidden_layer_sizes=(500,500,500,),
                        tol=1e-8,
                        learning_rate='adaptive',
                        learning_rate_init=0.001,
                        random_state=1,
                        early_stopping=False,
                        validation_fraction=0.2)

regressor = regressor.fit(X_train, y_train)

In [21]:
print "Train data: score= {}".format(regressor.score(X_train, y_train))
print "Test data: score= {}".format(regressor.score(X_test, y_test))
print "Validate data: score= {}".format(regressor.score(X_valid, y_valid))

Train data: score= 0.898270116465
Test data: score= 0.889897784567
Validate data: score= 0.48969144404
